### 전문가 정보 수집

In [1]:
import pandas as pd

orig_expert_df = pd.read_csv('/home/ys0660/happycat/expert_answer_urls.csv')

In [2]:
import requests
from lxml import html
import time

# 기본 URL
base_url = "https://kin.naver.com/userinfo/expert/answerList.naver?u=Gv4p7DdSVJtZzjT0OeHujZ6g%2BNwPV0h6qaN%2FpjLdiwU%3D"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

# --- 1️⃣ 전문가 기본 정보 수집 ---
response = requests.get(base_url, headers=headers)
response.raise_for_status()
tree = html.fromstring(response.content)

expert_name = tree.xpath('string(//*[@id="container"]/div/div/div[1]/div[2]/div[2]/div/dl/dt)')
expert_job = tree.xpath('string(//*[@id="container"]/div/div/div[1]/div[2]/div[2]/div/dl/dd/span)')
expert_position = tree.xpath('string(//*[@id="container"]/div/div/div[1]/div[2]/div[2]/div/p)')

print("👤 전문가 정보")
print("이름:", expert_name.strip())
print("직업:", expert_job.strip())
print("소속/직책:", expert_position.strip())

# --- 2️⃣ 답변 URL 수집 ---
answer_urls = []
page = 1

while True:
    url = f"{base_url}&page={page}"
    print(f"📄 페이지 {page} 수집 중...")

    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        break

    tree = html.fromstring(res.content)
    links = tree.xpath('//*[@id="au_board_list"]//a[@rel="KIN"]/@href')

    if not links:  # 더 이상 결과가 없으면 종료
        break

    # 상대경로 → 절대경로로 변환
    full_links = ["https://kin.naver.com" + link for link in links]
    answer_urls.extend(full_links)

    # polite crawling
    time.sleep(0.5)
    page += 1

print(f"\n✅ 총 {len(answer_urls)}개의 답변 URL 수집 완료")
print(answer_urls[:5])  # 일부 확인용

# --- 3️⃣ CSV로 저장 (선택) ---
import pandas as pd
df = pd.DataFrame({
    "expert_name": expert_name.strip(),
    "expert_job": expert_job.strip(),
    "expert_position": expert_position.strip(),
    "answer_url": answer_urls
})

df_update = pd.concat([orig_expert_df, df], ignore_index=True)

df_update.to_csv("expert_answer_urls.csv", index=False, encoding="utf-8-sig")

👤 전문가 정보
이름: 윤성진
직업: 수의사
소속/직책: 이리온 송파점 원장
📄 페이지 1 수집 중...
📄 페이지 2 수집 중...
📄 페이지 3 수집 중...
📄 페이지 4 수집 중...
📄 페이지 5 수집 중...
📄 페이지 6 수집 중...
📄 페이지 7 수집 중...
📄 페이지 8 수집 중...
📄 페이지 9 수집 중...
📄 페이지 10 수집 중...
📄 페이지 11 수집 중...
📄 페이지 12 수집 중...
📄 페이지 13 수집 중...
📄 페이지 14 수집 중...
📄 페이지 15 수집 중...
📄 페이지 16 수집 중...
📄 페이지 17 수집 중...
📄 페이지 18 수집 중...
📄 페이지 19 수집 중...
📄 페이지 20 수집 중...
📄 페이지 21 수집 중...
📄 페이지 22 수집 중...
📄 페이지 23 수집 중...
📄 페이지 24 수집 중...
📄 페이지 25 수집 중...
📄 페이지 26 수집 중...
📄 페이지 27 수집 중...
📄 페이지 28 수집 중...
📄 페이지 29 수집 중...
📄 페이지 30 수집 중...
📄 페이지 31 수집 중...
📄 페이지 32 수집 중...
📄 페이지 33 수집 중...
📄 페이지 34 수집 중...
📄 페이지 35 수집 중...
📄 페이지 36 수집 중...
📄 페이지 37 수집 중...
📄 페이지 38 수집 중...
📄 페이지 39 수집 중...
📄 페이지 40 수집 중...
📄 페이지 41 수집 중...
📄 페이지 42 수집 중...
📄 페이지 43 수집 중...
📄 페이지 44 수집 중...
📄 페이지 45 수집 중...
📄 페이지 46 수집 중...
📄 페이지 47 수집 중...
📄 페이지 48 수집 중...
📄 페이지 49 수집 중...
📄 페이지 50 수집 중...
📄 페이지 51 수집 중...
📄 페이지 52 수집 중...
📄 페이지 53 수집 중...
📄 페이지 54 수집 중...
📄 페이지 55 수집 중...
📄 페이지 56 수집 중...
📄 페이지 57 수집 중.

### 전문가의 질문-답변 수집

In [3]:
import requests
from lxml import html
import pandas as pd
import time
import os

# ===== 1️⃣ URL 목록 불러오기 =====
df_urls = pd.read_csv("expert_answer_urls.csv")
urls = df_urls["answer_url"].tolist()

# ===== 2️⃣ 기존에 수집된 URL 불러오기 =====
existing_file = "naver_kin_qna_detail.csv"

if os.path.exists(existing_file):
    df_existing = pd.read_csv(existing_file)
    existing_urls = set(df_existing["url"].tolist())
    print(f"📂 기존 파일에서 {len(existing_urls)}개 URL 확인됨.")
else:
    df_existing = pd.DataFrame(columns=["url", "title", "question", "answer"])
    existing_urls = set()
    print("📂 기존 파일이 없습니다. 새로 생성합니다.")

# ===== 3️⃣ 새로 수집할 URL만 필터링 =====
new_urls = [u for u in urls if u not in existing_urls]
print(f"🆕 새로 수집할 URL: {len(new_urls)}개")

# ===== 4️⃣ User-Agent 헤더 =====
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

# ===== 5️⃣ 결과 저장용 리스트 =====
data = []

for i, url in enumerate(new_urls, start=1):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        tree = html.fromstring(response.content)

        # XPath 추출
        part1 = tree.xpath('//*[@id="content"]/div[1]/div[1]/div[1]/text()')
        part2 = tree.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/text()')
        part3 = tree.xpath('//*[@id="content"]/div[1]//div[contains(@id, "SE-")]/div/div/div//text()')

        # 문자열 정제
        title = " ".join([t.strip() for t in part1 if t.strip()])
        question = " ".join([t.strip() for t in part2 if t.strip()])
        answer = " ".join([t.strip() for t in part3 if t.strip()])

        data.append({
            "url": url,
            "title": title,
            "question": question,
            "answer": answer
        })

        print(f"✅ [{i}/{len(new_urls)}] 수집 완료: {title[:30]}...")

    except Exception as e:
        print(f"❌ [{i}/{len(new_urls)}] 실패: {url} ({e})")
        continue

    # 서버 부담 완화
    time.sleep(0.5)

# ===== 6️⃣ 새 데이터 DataFrame 변환 =====
df_new = pd.DataFrame(data)

# ===== 7️⃣ 기존 데이터와 합치기 & 중복 제거 =====
df_combined = pd.concat([df_existing, df_new], ignore_index=True)
df_combined.drop_duplicates(subset=["url"], inplace=True)

# ===== 8️⃣ CSV 저장 =====
df_combined.to_csv(existing_file, index=False, encoding="utf-8-sig")
print(f"💾 {existing_file} 업데이트 완료 ({len(df_combined)}개 데이터)")


📂 기존 파일에서 4839개 URL 확인됨.
🆕 새로 수집할 URL: 3960개
✅ [1/3960] 수집 완료: 강아지 허벅지 안쪽 쥐젖?...
✅ [2/3960] 수집 완료: 강아지가 이상한 소리를내요 제발도와주세요...
✅ [3/3960] 수집 완료: 저희 집 강아지가 이상해요...
✅ [4/3960] 수집 완료: 3개월 된 새끼 고양이 투명한 점액 변...
✅ [5/3960] 수집 완료: 2개월 강아지 설사...
✅ [6/3960] 수집 완료: 3개월 된 아기 고양이 사료 토...
✅ [7/3960] 수집 완료: 강아지 이게 뭔가요 ㅜㅜ...
✅ [8/3960] 수집 완료: 강아지 점액변...
✅ [9/3960] 수집 완료: 강아지가 똥꼬스키 하면서 항문에서 이상한게 계속 나와요...
✅ [10/3960] 수집 완료: 개코 도마뱀 성별 문의...
✅ [11/3960] 수집 완료: 고양이 토  ㅠㅠ 색깔이.. 병원가야할까요?...
✅ [12/3960] 수집 완료: 강아지 기운없어요....
✅ [13/3960] 수집 완료: 강아지 중성화수술...
✅ [14/3960] 수집 완료: 수의사가 꿈인데 얼마나 노력해야 할까요?...
✅ [15/3960] 수집 완료: 젖먹이는 고양이 발정 가능한가요?...
✅ [16/3960] 수집 완료: 강아지가 토를 했는데 벌레가 나왔어요...
✅ [17/3960] 수집 완료: 강아지 배에 혹이 있어요ㅠㅠ...
✅ [18/3960] 수집 완료: 고양이 중성화하면 얌전해지나요?...
✅ [19/3960] 수집 완료: 중형견이 샤인머스켓/포도를 먹었어요...
✅ [20/3960] 수집 완료: 강아지 화이트초코렛...
✅ [21/3960] 수집 완료: 고양이 사료 토 오늘 2번,,,,지금 병원 가야 할까요...
✅ [22/3960] 수집 완료: 강아지 관련 질문...
✅ [23/3960] 수집 완료: 고양이 머리끈 삼킨 것 같아요...
✅ [24/3960] 수집 완료: 강아지 기침? 숨소리? 이상한데 무슨 질병인가요?...
✅ [25/